# core

> Functions to make the use of Monday.com even easier for Jelle.

In [1]:
# | default_exp core

In [2]:
# | hide
from nbdev.showdoc import *

In [3]:
# | export
import requests
import json
import os
from dotenv import load_dotenv
from textwrap import dedent

## Load the environment variables and set the API headers

In [4]:
# | export
load_dotenv()
apiKey = os.environ["API_KEY_MONDAY"]
apiUrl = "https://api.monday.com/v2"
headers = {"Authorization": f"Bearer {apiKey}", "API-Version": "2023-04"}

## Some example json queries

#### Boards

| Boardname | Board id | 
| --- | --- |
| Epic | 757753649 |
| Sprint bord, actief | 757790388 |

#### Columns Epic board

#### Columns Sprint board

| Column Text | Column ID | Type | Value text | Value id |
| --- | --- | --- | --- | --- |
| Status stories | status_stories | Select | To Do | 16 |
| " | " | " | Working on it | 0 |
| " | " | " | Wacht op Antwoord | 6 |
| " | " | " | Wacht op review | 7 |
| " | " | " | Done | 1 |
| Due Date | date7 | ? | ? | ? |
| Trekker | person | ? | ? | assigned_to_me |
| Reviewer | people | ? | ? | assigned_to_me |




Query to fetch all tasks for current sprint assigned to Jelle

In [34]:
# | export
query_sprint_tasks = dedent("""
    query {
        boards(ids: 757790388) {
            items_page(
                query_params: {
                    rules: [
                        {
                        column_id: "status_stories",
                        compare_value: [16, 0, 6, 7], #  ["To Do", "Working on it", "Wacht op Antwoord", "Wacht op review"],
                        operator: any_of
                    }, {
                        column_id: "date7",
                        compare_value: ["EXACT", "2025-04-23"],
                        operator: lower_than_or_equal
                    }],
                    operator: and
                    groups: {
                        rules: [
                            { column_id: "person",
                            compare_value: ["assigned_to_me"],
                            operator: any_of
                            },
                            { column_id: "people",
                            compare_value: ["assigned_to_me"],
                            operator: any_of
                            }
                        ],
                        operator: or
                        }
                    },
            ) {
                cursor
                items {
                    id
                    name
                    column_values(ids: ["person", "people", "status_stories", "numbers5", "date7", "story_syntax"]) {
                    id
                    text
                    value
                    }
                }
            }
        }
    }
    """).strip()

See all column names and their ids from a given board

In [17]:
# | export
query_board_columns = dedent("""
    query {
        boards(
            ids: [757790388]
        ) {
            columns {
                id
                title
                type
            }
        }
    }""").strip()

## Retrieve the data from Monday.com

In [31]:
# | export
def get_data(
    query: str,  # GraphQL query
    api_url: str = apiUrl,  # Monday.com API url
    headers: dict = headers,  # Monday.com API headers
) -> dict:  # response from Monday.com API
    data = {"query": query}
    return requests.post(url=apiUrl, json=data, headers=headers)

In [32]:
m_cols = get_data(query_board_columns)
m_cols.json()

{'data': {'boards': [{'columns': [{'id': 'name',
      'title': 'Name',
      'type': 'name'},
     {'id': 'person', 'title': 'Trekker', 'type': 'people'},
     {'id': 'people', 'title': 'Reviewer', 'type': 'people'},
     {'id': 'status_stories', 'title': 'Status stories', 'type': 'status'},
     {'id': 'label', 'title': 'Taak type', 'type': 'status'},
     {'id': 'numbers5', 'title': 'Estimation', 'type': 'numbers'},
     {'id': 'link_to_stories__main2',
      'title': 'Epic',
      'type': 'board_relation'},
     {'id': 'mirror__1', 'title': 'STP epic 1e schatting', 'type': 'mirror'},
     {'id': 'mirror6__1',
      'title': 'STP epic huidige schatting',
      'type': 'mirror'},
     {'id': 'subitems', 'title': 'Subitems', 'type': 'subtasks'},
     {'id': 'mirror_10', 'title': 'Status epic', 'type': 'mirror'},
     {'id': 'creation_log', 'title': 'Date created', 'type': 'creation_log'},
     {'id': 'date7', 'title': 'Due Date', 'type': 'date'},
     {'id': 'date', 'title': 'Done Dat

In [35]:
# | export
r = get_data(query_sprint_tasks)
r.json()

{'data': {'boards': [{'items_page': {'cursor': None,
     'items': [{'id': '1901240058',
       'name': 'Verwerken opmerkingen Stefan in document voorstel Python ondersteuning',
       'column_values': [{'id': 'person',
         'text': 'Jelle de Jong',
         'value': '{"changed_at":"2025-04-04T08:11:20.725Z","personsAndTeams":[{"id":23028787,"kind":"person"}]}'},
        {'id': 'people', 'text': '', 'value': None},
        {'id': 'status_stories',
         'text': 'Wacht op review',
         'value': '{"index":7,"post_id":null,"changed_at":"2025-04-14T11:43:07.035Z"}'},
        {'id': 'numbers5', 'text': '1', 'value': '"1"'},
        {'id': 'date7',
         'text': '2025-04-14',
         'value': '{"date":"2025-04-14","changed_at":"2025-04-04T08:13:10.724Z"}'},
        {'id': 'story_syntax', 'text': '', 'value': None}]},
      {'id': '1866097158',
       'name': 'Richtlijnen Databricks Code uitwerken',
       'column_values': [{'id': 'person',
         'text': 'Joris Monster',
   

## Convert the API response to Markdown

Het json deel van de response van de API query wordt geconverteerd naar Markdown.
Het format van de response en de gewenste gegevens bepalen natuurlijk hoe je die gegevens ophaalt.
De functie `get_items` pakt dit op een lompe, maar voor mij, bruikbare manier aan.

Stel je hebt de volgende response:
```json
{'data': 
    {'items_page_by_column_values': 
        {'items': [ 
            {'id': '1720037548',
            'name': 'Maak een voorbeeld van een Planning Bord en bespreek met Datalab',
            'column_values': [
                {'id': 'status_stories', 'text': 'Wacht op review'},
                {'id': 'numbers5', 'text': '2'},
                {'id': 'date7', 'text': '2025-02-19'},
                {'id': 'story_syntax', 'text': ''}]},
            {'id': '1535950975',
            'name': 'Doorgeven aan Alien: W-connect pagina vervallen links er uit halen',
            'column_values': [
                {'id': 'status_stories', 'text': 'To Do'},
                {'id': 'numbers5', 'text': '1'},
                {'id': 'date7', 'text': '2025-02-19'},
                {'id': 'story_syntax', 'text': ''}]}
        ]
        }
    }
}
```
Je wil de 'items' hebben, dus 'data', 'items_page_by_column_values' en 'items' zelf moet je als het ware overslaan om de benodigde list te krijgen.

Dus de functie roep je zo aan.

```python
get_items(request_response, ['data','items_page_by_column_values', 'items'])
```


In [36]:
# | export
def get_items(
    response: requests.models.Response,  # response from Monday.com API
    skip_keys: list = ["data", "items_page_by_column_values", "items"],  # list of key values in the response to skip
) -> list:  # list of items
    return [item for item in response.json().values() if not any(key in item for key in skip_keys)]

In [59]:
# | export
def item_to_markdown(
    item: str,  # json formatted string response from Monday.com API
) -> str:  # Task formatted as Markdown for Obsidian
    """Convert Monday.com item to Markdown task format"""
    # Extract the basic fields
    name = item["name"]

    # Extract role: reviewer or trekker
    
    for col in item["column_values"]:
        if col["id"] == "people" and col["text"] == "Jelle de Jong":
            role = " 🤝reviewer"
            break # break after finding the role
        elif col["id"] == "person" and col["text"] == "Jelle de Jong":
            role = "" 
            break # break after finding the role
        
    # Extract values from column_values
    for col in item["column_values"]:
        if col["id"] == "numbers5":
            try:
                duration = int(col["text"]) * 120
            except ValueError:
                duration = 0
        elif col["id"] == "date7":
            date = col["text"]

    # Build the markdown string
    markdown = f"- [ ] {name}{role}"

    # Add duration if present
    if duration > 0:
        markdown += f" [{duration}m]"

    # Add priority emoji and date if present
    markdown += " ⏫"  # Priority marker
    if date:
        markdown += f" 📅 {date}"

    return markdown

In [60]:
# | export

for item in r.json()["data"]["boards"][0]["items_page"]["items"]:
    print(item_to_markdown(item))

- [ ] Verwerken opmerkingen Stefan in document voorstel Python ondersteuning [120m] ⏫ 📅 2025-04-14
- [ ] Richtlijnen Databricks Code uitwerken 🤝reviewer [120m] ⏫ 📅 2025-04-23
- [ ] Vergelijkbare periode opzoeken in de 3 jaar met elk van de 3 methodes 🤝reviewer [240m] ⏫ 📅 2025-04-23
- [ ] Statistische tests uitvoeren 🤝reviewer [360m] ⏫ 📅 2025-04-23
- [ ] Intern overleg: Hoe kunnen we hydrologen en anderen buiten datalab helpen om Python code te schrijven, beherenen delen [120m] ⏫ 📅 2025-04-23
- [ ] Validatie tool monitoringsgegevens begrijpen en draaien [120m] ⏫ 📅 2025-04-23
- [ ] Maximum toerental van gemalen uit Clearscada implementeren in code 🤝reviewer [240m] ⏫ 📅 2025-04-23


In [13]:
# | hide
import nbdev

nbdev.nbdev_export()